# Training Model with Representation Tracking

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
BASE_DIR = "drive/MyDrive/CS159/"

In [4]:
import os
import sys
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import shape as s
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout


In [5]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [6]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPU


In [6]:
# Define parameters of some images
DIM = 227

In [7]:
class AlexNet(Sequential):
    '''
    Here is an implementation of AlexNet in Tensorflow.
    I have a lot of experience with AlexNet and it works pretty
    well for most things. We can train from scratch or we can get the 
    pretrained ImageNet weights from the Internet and do stuff with it

    We can also use networks ike VGG16 and ResNet (I think ResNet is classification)
    and do similar things, we can just get those implemnetations there. 
    '''
    def __init__(self, input_shape, num_classes):
        super().__init__()

        self.add(Conv2D(96, kernel_size=(11, 11), strides=4,
                        padding='valid', activation='relu',
                        input_shape=input_shape,
                        kernel_initializer='he_normal'))
        self.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2),
                              padding='valid', data_format=None))

        self.add(Conv2D(256, kernel_size=(5, 5), strides=1,
                        padding='same', activation='relu',
                        kernel_initializer='he_normal'))
        self.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2),
                              padding='valid', data_format=None))

        self.add(Conv2D(384, kernel_size=(3, 3), strides=1,
                        padding='same', activation='relu',
                        kernel_initializer='he_normal'))

        self.add(Conv2D(384, kernel_size=(3, 3), strides=1,
                        padding='same', activation='relu',
                        kernel_initializer='he_normal'))

        self.add(Conv2D(256, kernel_size=(3, 3), strides=1,
                        padding='same', activation='relu',
                        kernel_initializer='he_normal'))

        self.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2),
                              padding='valid', data_format=None))

        self.add(Flatten())
        self.add(Dense(4096, activation='relu'))
        self.add(Dropout(0.5))
        self.add(Dense(4096, activation='relu'))
        self.add(Dropout(0.5))
        self.add(Dense(num_classes, activation='softmax'))


In [8]:
class AlexNet_Input_to_Kernels(Sequential):
    '''
    H
    '''
    def __init__(self, input_shape):
        super().__init__()

        self.add(Conv2D(96, kernel_size=(11, 11), strides=4,
                        padding='valid', activation='relu',
                        input_shape=input_shape,
                        kernel_initializer='he_normal'))
        self.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2),
                              padding='valid', data_format=None))

        self.add(Conv2D(256, kernel_size=(5, 5), strides=1,
                        padding='same', activation='relu',
                        kernel_initializer='he_normal'))
        self.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2),
                              padding='valid', data_format=None))

        self.add(Conv2D(384, kernel_size=(3, 3), strides=1,
                        padding='same', activation='relu',
                        kernel_initializer='he_normal'))

        self.add(Conv2D(384, kernel_size=(3, 3), strides=1,
                        padding='same', activation='relu',
                        kernel_initializer='he_normal'))

        self.add(Conv2D(256, kernel_size=(3, 3), strides=1,
                        padding='same', activation='relu',
                        kernel_initializer='he_normal'))

        self.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2),
                              padding='valid', data_format=None))
        self.add(Flatten(input_shape=(6, 6, 256)))
        # self.add(Dense(4096, activation='relu'))
#         self.add(Dropout(0.5))
#         self.add(Dense(4096, activation= 'relu'))
#         self.add(Dropout(0.5))


In [28]:
DIM = 227
img_size = (DIM, DIM)
image_height = DIM
image_width = DIM

IMG_FOLDER_PATH = BASE_DIR + "/data"# Add training set here
# IMG_VAL_PATH = ""# Add validation set here
# IMG_TEST_PATH = ""# Add test set here
data_generator = ImageDataGenerator(rescale=1.0/255.0)
data_iter = tf.keras.utils.image_dataset_from_directory(IMG_FOLDER_PATH,
                                                labels='inferred',
                                                label_mode='categorical' ,
                                                class_names=['2s', '4s', 'ch', 'cu'],
                                                color_mode='rgb',
                                                batch_size=1,
                                                image_size=(DIM, DIM),
                                                shuffle=True,
                                                seed=1,
                                                validation_split=0.1,
                                                subset="training",
                                                interpolation='bilinear',
                                                follow_links=False,
                                                crop_to_aspect_ratio=False

                                               )
val_data_iter = tf.keras.utils.image_dataset_from_directory(IMG_FOLDER_PATH,
                                                labels='inferred',
                                                label_mode='categorical' ,
                                                class_names=['2s', '4s', 'ch', 'cu'],
                                                color_mode='rgb',
                                                batch_size=1,
                                                image_size=(DIM, DIM),
                                                shuffle=True,
                                                seed=1,
                                                validation_split=0.1,
                                                subset="validation",
                                                interpolation='bilinear',
                                                follow_links=False,
                                                crop_to_aspect_ratio=False

                                               )
# val_data_iter = data_generator.flow_from_directory(IMG_VAL_PATH,
#                                                    target_size=img_size,
#                                                    color_mode="rgb",
#                                                    batch_size=1,
#                                                    shuffle=False,
#                                                    class_mode="categorical"
#                                                    )

# test_data_iter = data_generator.flow_from_directory(IMG_TEST_PATH,
#                                                     target_size=img_size,
#                                                     color_mode="rgb",
#                                                     batch_size=1,
#                                                     shuffle=False,
#                                                     class_mode="categorical"
#                                                     )


Found 567 files belonging to 4 classes.
Using 511 files for training.
Found 567 files belonging to 4 classes.
Using 56 files for validation.


In [29]:
labels = data_iter.
top_0 = max(np.where(labels == 0)[0])
top_1 = max(np.where(labels == 1)[0])
top_2 = max(np.where(labels == 2)[0])
top_3 = max(np.where(labels == 3)[0])

labels[top_0]

SyntaxError: ignored

In [30]:
# We will build a custom training loop to see how the process works
# we compile it, load the weights
model = AlexNet((DIM, DIM, 3), 4)
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

Model: "alex_net_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_145 (Conv2D)         (None, 55, 55, 96)        34944     
                                                                 
 max_pooling2d_87 (MaxPoolin  (None, 27, 27, 96)       0         
 g2D)                                                            
                                                                 
 conv2d_146 (Conv2D)         (None, 27, 27, 256)       614656    
                                                                 
 max_pooling2d_88 (MaxPoolin  (None, 13, 13, 256)      0         
 g2D)                                                            
                                                                 
 conv2d_147 (Conv2D)         (None, 13, 13, 384)       885120    
                                                                 
 conv2d_148 (Conv2D)         (None, 13, 13, 384)       1

In [31]:
def visualize_representation(input, title=""):
    #input = np.reshape(input, (len(input), 1))
    for i in [60, 200, 350, 550]:
        val = input[i]
        plt.figure()
        plt.bar(x=[i for i in range(1, 4097)], height=val)
        plt.title(title)
        plt.tight_layout()
        plt.show()

In [33]:
# We will build a custom training loop to see how the process works
# we compile it, load the weights
model = AlexNet((DIM, DIM, 3), 4)
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss="categorical_crossentropy", metrics=["accuracy"])

EPOCHS = 5 * 2
layers = [layer.name for layer in model.layers]

iteration_representations = {}
for epoch in range(EPOCHS):
    print(f"Iteration {epoch + 1}")
    hist = model.fit(data_iter, epochs=1, batch_size=32, validation_data=val_data_iter)

    layer_cut = 9

    model1 = AlexNet_Input_to_Kernels((DIM, DIM, 3))

    for i in range(0, layer_cut):
        model1.layers[i].set_weights(model.layers[i].get_weights())

    # Now we can take the outputs from the cut model

    outputs = model1.predict(data_iter)

    print(f"Length of Outputs: {outputs.shape}")
    
    iteration_representations[epoch + 1] = outputs

    #visualize_representation(outputs)

Iteration 1
511/511 [==============================] - 12s 22ms/step - loss: 72.4405 - accuracy: 0.3659 - val_loss: 1.2470 - val_accuracy: 0.3571
Length of Outputs: (511, 9216)
Iteration 2
511/511 [==============================] - 11s 21ms/step - loss: 1.2730 - accuracy: 0.5695 - val_loss: 1.0486 - val_accuracy: 0.5714
Length of Outputs: (511, 9216)
Iteration 3
511/511 [==============================] - 11s 21ms/step - loss: 0.9842 - accuracy: 0.6888 - val_loss: 0.5930 - val_accuracy: 0.8214
Length of Outputs: (511, 9216)
Iteration 4
511/511 [==============================] - 11s 21ms/step - loss: 1.0029 - accuracy: 0.6967 - val_loss: 1.3971 - val_accuracy: 0.6071
Length of Outputs: (511, 9216)
Iteration 5
511/511 [==============================] - 11s 21ms/step - loss: 0.4342 - accuracy: 0.8669 - val_loss: 0.5527 - val_accuracy: 0.7857
Length of Outputs: (511, 9216)
Iteration 6
511/511 [==============================] - 11s 21ms/step - loss: 0.9399 - accuracy: 0.7299 - val_loss: 0.12

In [27]:
failed_imgs = []
for i, item in enumerate(data_iter):
    print(item)

(<tf.Tensor: shape=(32, 227, 227, 3), dtype=float32, numpy=
array([[[[236.      , 244.      , 246.      ],
         [235.18723 , 244.18723 , 243.18723 ],
         [237.      , 246.      , 245.      ],
         ...,
         [146.64539 , 172.64539 , 197.64539 ],
         [145.25337 , 169.25337 , 195.25337 ],
         [141.35931 , 168.81732 , 193.08832 ]],

        [[236.      , 244.      , 246.      ],
         [235.18723 , 244.18723 , 243.18723 ],
         [236.      , 245.      , 244.      ],
         ...,
         [146.64539 , 172.64539 , 197.64539 ],
         [144.37451 , 168.37451 , 194.37451 ],
         [141.54199 , 169.      , 193.271   ]],

        [[235.      , 243.      , 245.      ],
         [235.49232 , 244.49232 , 243.49232 ],
         [236.      , 244.60573 , 244.78854 ],
         ...,
         [147.03966 , 173.03966 , 198.03966 ],
         [146.5077  , 170.5077  , 196.5077  ],
         [141.98357 , 169.44157 , 193.71257 ]],

        ...,

        [[112.46365 , 151.46365 